In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [4]:
block_size = 3 # context length: how many chars do we take to predict the next one

# build dataset
def build_dataset(words):
    
    X, Y = [], []
    for w in words:
#         print(w)

        context = [0]* block_size
        for ch in w+'.':
            idx = stoi[ch]
            X.append(context)
            Y.append(idx)
            context = context[1:] + [idx]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

In [120]:
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

X_train, Y_train = build_dataset(words[:n1])  #80%
X_eval, Y_eval = build_dataset(words[n1:n2])  #10%
X_test, Y_test = build_dataset(words[n2:])    #10%

torch.Size([182580, 3]) torch.Size([182580])
torch.Size([22767, 3]) torch.Size([22767])
torch.Size([22799, 3]) torch.Size([22799])


In [121]:
#Utility for comparing manual gradients to PyTorch Gradients
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [122]:
n_emb = 10
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_emb), generator = g) # 10 dimentional embedding
#Hidden Layer 
l1_n = 300
W1 = torch.randn((block_size*n_emb, n_hidden), generator = g) * (5/3)/(block_size*n_emb)**0.5 # Kaiming init
b1 = torch.randn(n_hidden, generator = g) * 0.1  #this is useless when using Batch Norm
# Second/Final layer
W2 = torch.randn((n_hidden, vocab_size), generator = g) * 0.1   # should not set weights as all 0, but a small number
b2 = torch.randn(vocab_size, generator = g) * 0.1    #biases can be 0 at the beginning
#BatchNorm Parameters
bngain = torch.ones((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.zeros((1, n_hidden)) * 0.1

# bnmean_running = torch.zeros((1,n_hidden))
# bnstd_running = torch.ones((1,n_hidden))

parameters = [C, W1, b1, W2, b2, bngain, bnbias]

print(sum(p.nelement() for p in parameters))

for p in parameters:
    p.requires_grad = True

4137


In [123]:
batch_size = 32
n = batch_size


In [124]:
idx = torch.randint(0, X_train.shape[0], (batch_size,), generator = g)
Xb,Yb = X_train[idx], Y_train[idx]

In [125]:
#Forward Pass
emb = C[Xb]  
embcat = emb.view(emb.shape[0],-1)

#Linear Layer
hprebn= embcat  @ W1 + b1
#BatchNorm Layer
bnmeani = 1/n* hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff ** 2

bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5) **-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

# Non-linearity
h = torch.tanh(hpreact)

# Linear layer 2
logits = h @ W2 +b2 #output layer

#cross entropy loss
logit_maxes = logits.max(1, keepdim = True).values
norm_logits = logits - logit_maxes
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims = True)
counts_sum_inv = counts_sum **-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = - logprobs[range(n), Yb].mean()

#PyTorch backward pass
for p in parameters:
    p.grad = None

for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logit_maxes, logits,h, hpreact, bnraw,
          bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
          embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.5116, grad_fn=<NegBackward0>)

In [126]:
logprobs[range(n), Yb]

tensor([-3.6375, -3.8762, -2.9398, -3.7223, -3.1410, -3.0782, -3.4667, -3.0483,
        -2.9206, -3.9202, -3.2367, -3.5328, -3.1390, -3.4396, -4.2627, -4.3207,
        -2.9114, -2.9832, -4.8319, -3.5892, -3.7630, -2.7795, -3.4824, -3.9773,
        -3.4955, -3.1612, -4.0071, -3.3665, -3.2455, -4.3827, -2.5700, -4.1426],
       grad_fn=<IndexBackward0>)

In [127]:
logprobs.shape

torch.Size([32, 27])

In [128]:
counts.shape, counts_sum_inv.shape

(torch.Size([32, 27]), torch.Size([32, 1]))

In [129]:
hpreact.shape, bnraw.shape, bnbias.shape

(torch.Size([32, 64]), torch.Size([32, 64]), torch.Size([1, 64]))

In [130]:
#Exercise 1

#loss = -1/3a + -1/3b + -1/3c
#dloss/da = -1/n

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n

#d(logx) =1/x 
dprobs = (1.0/probs) * dlogprobs #chain rule

dcounts_sum_inv = (counts * dprobs).sum(1, keepdim = True) 
dcounts = counts_sum_inv*dprobs  #first contribution

dcounts_sum = -(counts_sum**-2) * dcounts_sum_inv
dcounts += torch.ones_like(counts) * dcounts_sum
# d e^x = e^x
dnorm_logits = counts * dcounts
dlogits = dnorm_logits.clone()
dlogit_maxes = - (dnorm_logits.sum(1, keepdim = True))
dlogits += F.one_hot(logits.max(1).indices, num_classes = logits.shape[1]) * dlogit_maxes

dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits .sum(0)

dhpreact = (1.0 - h**2)*dh

dbngain = (bnraw * dhpreact).sum(0, keepdim = True)
dbnraw = bngain * dhpreact
dbnbias = dhpreact.sum(0, keepdim = True)

dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim = True)
dbndiff = bnvar_inv *dbnraw

dbnvar = -0.5*(bnvar + 1e-5)**-1.5*dbnvar_inv
dbndiff2 =  1.0/(n-1.0) * torch.ones_like(bndiff2) * dbnvar
dbndiff += 2 * bndiff * dbndiff2

dhprebn = dbndiff.clone()
dbnmeani = - dbndiff.sum(0, keepdim =True)
dhprebn+= 1.0/n * torch.ones_like(hprebn) * dbnmeani

dembcat = dhprebn @ W1.T 
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)

demb = dembcat.view(emb.shape)

dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        idx = Xb[k,j]
        dC[idx] += demb[k,j]
cmp('log probs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum_', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2',dW2, W2)
cmp('b2',db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnraw', dbnraw, bnraw)
cmp('dbnbias', dbnbias, bnbias)
cmp('dbnvar_inv', dbnvar_inv, bnvar_inv)
cmp('dbnvar', dbnvar, bnvar)
cmp('dbndiff2', dbndiff2, bndiff2)
cmp('dbndiff', dbndiff, bndiff)
cmp('dbnmeani', dbnmeani, bnmeani)

cmp('dhprebn', dhprebn, hprebn)
cmp('dembcat', dembcat, embcat)
cmp('dW1', dW1, W1)
cmp('db1', db1, b1)
cmp('demb', demb, emb)
cmp('dC', dC, C)
# h.shape, W2.shape, logits.shape

log probs       | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_     | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff: 0.0
dbnbias         | exact: True  | approximate: True  | maxdiff:

In [131]:
#Exercise 2
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff', (loss_fast - loss).item())

3.511599063873291 diff 0.0


In [132]:
logits.shape, loss.shape

(torch.Size([32, 27]), torch.Size([]))

In [133]:
dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits/= n

cmp('logits', dlogits, logits)

logits          | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09


In [134]:
#Exercise 3
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim = True)) / torch.sqrt(hprebn.var(0, keepdim= True, unbiased=True) + 1e-5) + bnbias
print('max diff',  (hpreact_fast - hpreact).abs().max())

max diff tensor(4.7684e-07, grad_fn=<MaxBackward1>)
